# Remote Data Access and Units
Ultimately we want to make a Skew-T plot in Python using MetPy, but we'll need to learn a bit about a few modules including **siphon, numpy, matplotlib, and MetPy** to get us to that end. This notebook will start with reading data from a remote web server and talking about units with arrays.

Siphon helps us access remote data (buoy data, surface station archives, climatology data, etc.)

We will read in data generated from the University of Wyoming and slightly modified

http://weather.uwyo.edu/upperair/sounding.html

The main difference between the output from this site and reading them in below is removing/modifying lines that have missing data. Remove lines at pressures below 100 mb that contain missing data; put in a missing value for levels below 100 mb (e.g., -999).

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
from metpy.units import units, pandas_dataframe_to_unit_arrays
from siphon.simplewebservice.wyoming import WyomingUpperAir

Siphon (another Unidata Python package) has a remote data access function to retrieve data from different types of archive data sources including soundings, National Data Bouy Center, and the ACIS station data archive. The example we'll do below uses the Wyoming sounding archive to retrieve a radiosonde launch data set.



## Get Station Data

Use siphon to access Wyoming Sounding archive:

Siphon Documentation: https://unidata.github.io/siphon/latest/api/simplewebservice.html#module-siphon.simplewebservice.wyoming

In this documentation, the WyomingUpperAir class has one method: request_data(). That method requires two things to get sounding data:
* time - format this using datetime()
* site_id - Can find via websites with soundings. One is the [RAP UCAR site](http://weather.rap.ucar.edu) and click on the Upper Air section. Note these stations have moved over time and issues can prevent a station launching a sounding on a particular day. Using the [Wyoming soundings](http://weather.uwyo.edu/upperair/sounding.html) site, you can find what stations were available on any particular day and where they were located.

In [ ]:
# Need these to get upper air data.
date = datetime(2005,1,11,12)
station = 'OUN'

In [ ]:
df = WyomingUpperAir.request_data(date, station)

## Pull out data from dataset variable

The above function essentially gives us a Pandas DataFrame that has an assortment of different data

In [ ]:
df

# Can subset the data to just the data we want.
#temperature = df.temperature

# But notice it has no units associated with it.
print(temperature)

## Units!

Units are exceedingly important in meteorology (and all sciences). With the file we have read in from the Wyoming Sounding archive, we also can get a dictionary of the unit values for each of the variables. Let's first just print out the unit values associated with each variable.

Fortunately, Siphon packages our data with a dictionary of units. Access this with `df.units`.

Pandas does not support attaching units to data, but metpy has been coded with a function to quickly attach these units to all the variables.
```python
metpy.units.pandas_dataframe_to_unit_arrays(<pandas dataframe>)
```
This results in a dictionary filled with arrays with units.

In [ ]:
df.units

## Pulling out data and attaching Units

The dataset object is a standard format dictionary that many different data read functions produce. To access a variable you'll reference the dataset variable name (dataset) with selecting the variable from the directionary as a string.

Use MetPy function (`pandas_dataframe_to_unit_arrays`) to pull data from a Pandas DataFrame and make numpy arrays with attached units for use in MetPy calculations.

In [ ]:
# Attach the units!
data_with_units = pandas_dataframe_to_unit_arrays(df)

In [ ]:
data_with_units
#temperature_with_units = data_with_units['temperature']


In [ ]:
#print(temperature)
#print(temperature_with_units)

In [ ]:
# Only want to pull in data up to 100 mb, we can 
# use a boolean array to choose on the desired values
pressure = data_with_units['pressure']
print(pressure)

# Create boolean object for where pressure is greater than 100hPa
index_pressure_100 = pressure >= 100 * units.hPa
print(index_pressure_100)

# Subset pressure and read in other data to only 100 mb
p = pressure[index_pressure_100]
T = temperature_with_units[index_pressure_100]
Td = data_with_units['dewpoint'][index_pressure_100]
z = data_with_units['height'][index_pressure_100]

In [ ]:
print(T)

## Plot Temperature vs. Height

In [ ]:
# Note matplotlib gives basic axis labels using the units in our data.
plt.plot(T, z)
#plt.plot(T.to('degF'), z.to('feet'))
#plt.plot(T.to('K'), z.to('feet'))

## Output to a file

It is not hard to write to a file, with a couple of simple steps we can accomplish the task.
1. Open a file for writing using `open()` function and assign it a variable name.
2. Use the variable name to use the `write()` method to write a single line. Must be a string format. Must manually put in a newline character `\n`.
3. Use a loop to print multiple lines of data a row at a time.
4. When doen writing all material, use the `close()` method to formally close the file and write to disk.

In [ ]:
# Open a file for writing
outfile = open('data_output.txt', 'w')

# Basic function to write text to files.
outfile.write('First line output to a file.')
outfile.write('Second line in file.')

# New Line
#outfile.write('First line output to a file.\n')
#outfile.write('Second line in file.')

# Variables
#outfile.write(T)
#outfile.write(f'{T}')
#outfile.write('TMPC\n')
#for i in range(T.shape[0]):
#   outfile.write(f'{T[i]}')
#   outfile.write(f'{T[i]}\n')
#   outfile.write(f'{T[i].m}\n')

# Free up memory by closing the file.
outfile.close()

In [ ]:
# Open a file for writing
outfile = open('data_output.txt', 'w')

# Variables
outfile.write(T)

#outfile.write(f'{T}')

#for i in range(T.shape[0]):
#   outfile.write(f'{T[i]}')
#   outfile.write(f'{T[i]}\n')

#outfile.write('TMPC\n')
#for i in range(T.shape[0]):
#   outfile.write(f'{T[i]}')
#   outfile.write(f'{T[i]}\n')
#   outfile.write(f'{T[i].m}\n')

# Free up memory by closing the file.
outfile.close()

In [ ]:
# Open a file for writing
outfile = open('data_output.txt', 'w')

outfile.write('PRES    TMPC\n')
for i in range(T.shape[0]):
    outfile.write(f'{p[i].m}  {T[i].m}\n')
    outfile.write(f'{p[i].m:7.2f}  {T[i].m:7.2f}\n')

outfile.close()